# Collaborative filtering

## import libraries

In [1]:
# Collaborative filteringimport pandas as pd
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate

## import files

In [2]:
input_path = 'Downloads/input/'

In [3]:
ratings_df = pd.read_csv(input_path + 'ratings.csv')
ratings_df = ratings_df.drop(columns = 'timestamp')

In [4]:
print(ratings_df.shape)
ratings_df.head()

(26024289, 3)


,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [5]:
links_df = pd.read_csv(input_path + 'links.csv')
links_df = links_df.drop(columns = 'imdbId')

In [6]:
print(links_df.shape)
links_df.head()

(45843, 2)


,movieId,tmdbId
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0


In [7]:
movies_df = pd.read_csv(input_path + 'tmdb_5000_movies.csv')
release_date = movies_df[['id', 'release_date']]

## collaborative filtering에 출시년도 정보를 담기 위해 release_date 데이터프레임을 추가

In [8]:
release_date['release_date'] = release_date['release_date'].apply(lambda x : str(x)[:4])
haha = links_df.join(release_date.set_index('id'), on='tmdbId')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
print(movies_df.shape)
movies_df.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


## TMDB 데이터셋 기반으로 영화 ID 리스트 생성

In [10]:
movieId_list = movies_df[['id', 'title']].set_index('id').join(links_df.set_index('tmdbId')[['movieId']]).dropna()
movieId_list['movieId'] = movieId_list['movieId'].map(int)
movieId_list = movieId_list.set_index('movieId')

In [11]:
movieId_list.shape

(4599, 1)

In [12]:
movieId_list.head()

,title
movieId,
18,Four Rooms
260,Star Wars
6377,Finding Nemo
356,Forrest Gump
2858,American Beauty


## TMDB 데이터셋에 존재하지 않는 영화에 대한 평점 삭제

In [13]:
ratings_df = ratings_df[ratings_df['movieId'].isin(movieId_list.index)]

In [14]:
ratings_df.shape

(17977131, 3)

## 적은 수의 평점을 받은 영화 삭제
- ### 최소 3774개의 평점 기록이 있는 영화만 쓰도록 함.

In [15]:
df_movie_summary = ratings_df.groupby('movieId')['rating'].agg(['count'])
movie_benchmark = int(round(df_movie_summary['count'].quantile(0.75),0))
movie_benchmark

3774

## 적은 수의 평점을 남긴 유저 삭제
- ### 최소 67개 이상의 리뷰를 남긴 유저의 데이터만을 사용하도록 함.

In [16]:
df_user_summary = ratings_df.groupby('userId')['rating'].agg(['count'])
user_benchmark = int(round(df_user_summary['count'].quantile(0.75),0))
user_benchmark

67

## 앞서 설정한 기준점을 바탕으로 평점 데이터셋 축소

In [17]:
original_movie_list = ratings_df['movieId'].unique()
original_user_count = ratings_df['userId'].nunique()

while True:
    df_movie_summary = ratings_df.groupby('movieId')['rating'].agg(['count'])
    drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index
    
    if len(drop_movie_list) == 0:
        break
    else:
        ratings_df = ratings_df[~ratings_df['movieId'].isin(drop_movie_list)]
    
    df_user_summary = ratings_df.groupby('userId')['rating'].agg(['count'])
    drop_user_list = df_user_summary[df_user_summary['count'] < user_benchmark].index
    
    if len(drop_user_list) == 0:
        break
    else:
        ratings_df = ratings_df[~ratings_df['userId'].isin(drop_user_list)]

drop_movie_list = list(set(original_movie_list) - set(ratings_df['userId'].unique()))

print("Dropped movie count : ", len(drop_movie_list))
print("Dropped user count : ", (original_user_count - ratings_df['userId'].nunique()))

Dropped movie count :  3567
Dropped user count :  208245


## 축소된 평점 데이터셋에서 한 userId가 남긴 평점 개수를 측정

In [18]:
ratings_df['count'] = ratings_df.groupby('userId')['rating'].transform('count')

## 평점 데이터셋 가공 후의 모습

In [19]:
print(ratings_df.shape)
ratings_df.head()

(10406807, 4)


,userId,movieId,rating,count
317,9,1,4.5,70
318,9,150,4.0,70
320,9,497,4.0,70
321,9,594,3.0,70
322,9,750,4.0,70


## 데이터 전처리 후 ratings dataframe에 남은 영화 종류의 총 수

In [20]:
# num of tmdb movies after data clean
ratings_df['movieId'].nunique()

985

In [21]:
# Data Example
print(ratings_df.iloc[::500000, :])

          userId  movieId  rating  count
317            9        1     4.5     70
1243663    12770    56174     4.0     81
2487283    25813     2359     4.0    307
3744890    39045     2161     4.5    319
4998745    51419     3114     4.0    125
6254815    64523     4034     1.5    303
7513976    77547     1644     3.0    146
8774176    90506     7147     1.0    181
10033527  103502      648     3.5    174
11296264  117194      858     5.0    129
12543779  130151     8784     5.0    226
13778426  143161      296     4.0     69
15016889  156186     1393     3.5    272
16248864  168830     1198     4.0    208
17488724  181477     1201     5.0     71
18734725  194606      527     4.5    141
19979490  207742      541     5.0    364
21225520  220483     5995     4.5    102
22481799  233410    48774     5.0     89
23741351  246455     3671     4.0    372
24996406  260013     1608     3.0    381


In [22]:
len(movieId_list)

4599

## 모델 성능평가

In [23]:
reader = Reader()
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
svd = SVD()

In [24]:
evaluation_result = cross_validate(svd, data)

In [25]:
evaluation_result = pd.DataFrame(evaluation_result)
evaluation_result.loc['mean'] = evaluation_result.mean()

In [26]:
evaluation_result[['test_rmse', 'test_mae']]

,test_rmse,test_mae
0,0.751656,0.568255
1,0.751891,0.568917
2,0.751226,0.568331
3,0.751548,0.568776
4,0.752203,0.569253
mean,0.751705,0.568707


## ratings dataframe을 이용하여 trainset 생성

In [27]:
# getting full dataset
data_set = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
train_data_set = data_set.build_full_trainset()

## 모델 학습

In [28]:
#model.fit(X_train, y_train).score(X_train, y_train)
%time svd.fit(train_data_set)

Wall time: 18min 10s


## 영화 리스트 재구성

In [29]:
result = movieId_list.copy()
result = result.reset_index()
result = result[result['movieId'].isin(ratings_df['movieId'])]
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 4449
Data columns (total 2 columns):
movieId    985 non-null int64
title      985 non-null object
dtypes: int64(1), object(1)
memory usage: 23.1+ KB


## 2000년도 이후 출시된 영화만 필터링

In [30]:
target_df = result[['movieId', 'title']]
k = haha[haha['movieId'].isin(target_df['movieId'])]
k = k.drop(columns=['tmdbId'])
Id_title_year = pd.merge(target_df, k, on='movieId', how='outer')
Id_title_year_over2000 = Id_title_year[Id_title_year['release_date'] >= str(2000)]
Id_title_year_over2000

,movieId,title,release_date
2,6377,Finding Nemo,2003
5,3910,Dancer in the Dark,2000
8,6539,Pirates of the Caribbean: The Curse of the Bla...,2003
9,6874,Kill Bill: Vol. 1,2003
12,54272,The Simpsons Movie,2007
13,7361,Eternal Sunshine of the Spotless Mind,2004
14,4235,Amores perros,2000
15,45722,Pirates of the Caribbean: Dead Man's Chest,2006
16,37733,A History of Violence,2005
18,5810,8 Mile,2002


## userId = 231이 평가한 영화의 수

In [31]:
ratings_df[ratings_df['userId'] == 231].nunique()

userId       1
movieId    607
rating       9
count        1
dtype: int64

## userId = 231이 과거에 평점 5점을 준 영화들

In [32]:
df_1 = ratings_df[(ratings_df['userId'] == 231) & (ratings_df['rating'] == 5.0)]
df_1 = df_1.set_index('movieId')
df_1 = df_1.join(movieId_list)['title']
print(len(df_1.index))
df_1

12


movieId
110                                             Braveheart
260                                              Star Wars
318                               The Shawshank Redemption
527                                       Schindler's List
1196                               The Empire Strikes Back
1198                               Raiders of the Lost Ark
1259                                           Stand by Me
2028                                   Saving Private Ryan
3578                                             Gladiator
4993     The Lord of the Rings: The Fellowship of the Ring
33166                                                Crash
60684                                             Watchmen
Name: title, dtype: object

## 예측 리스트에서 유저가 입력한 영화를 제거한 뒤 평점 예측 결과를 출력

In [33]:
def user_rating_prediction(userId, result, movietitle, recom_count=10):
    rating_prediction = result.copy()
    #print("length of result : ", len(result['movieId']))
    rating_prediction['Estimate_Score'] = rating_prediction['movieId'].apply(lambda x : svd.predict(userId, x).est)
    rating_prediction = rating_prediction.drop(columns='movieId')
    rating_prediction = rating_prediction[rating_prediction['title'] != movietitle]
    rating_prediction = rating_prediction.sort_values('Estimate_Score', ascending=False)
    rating_prediction = rating_prediction[rating_prediction['title'].isin(Id_title_year_over2000['title'])]

    return rating_prediction.head(recom_count)

In [34]:
user231_prediction = user_rating_prediction(231, result, 'Aliens', 44)
user231_prediction

,title,Estimate_Score
54,The Lord of the Rings: The Fellowship of the Ring,4.586857
55,The Lord of the Rings: The Two Towers,4.515533
56,The Lord of the Rings: The Return of the King,4.496908
41,Gladiator,4.043276
605,Star Wars: Episode III - Revenge of the Sith,3.985514
30,Memento,3.949355
543,Crash,3.895644
2938,Inception,3.894864
47,Snatch,3.883925
68,The Dark Knight,3.876589


## movietitle에 해당하는 영화에 평점 5점을 남긴 유저의 아이디 목록을 출력

In [35]:
def title2userlist(movietitle):
    movie_id = movieId_list.index[movieId_list['title'] == movietitle].tolist()[0]
    user_id_list = ratings_df[ratings_df['movieId'] == movie_id][ratings_df['rating'] == 5.0].sort_values('count', ascending=False)['userId'].values
    return user_id_list

## 위에 정의된 함수들을 통해 유저 100명에 해당하는 영화의 목록을 출력

In [36]:
def get_recommendation_by_title(movietitle, set_count=100, recom_count=10):
    user_id_list = title2userlist(movietitle)
    prediction_results = list()
    for user_id in user_id_list:
        if len(prediction_results) == set_count:
            break
        temp_list = user_rating_prediction(user_id, result, movietitle, recom_count)['title'].values
        if len(temp_list) == recom_count:
            prediction_results.append(temp_list)
    return prediction_results

## Aliens 영화를 좋아하는 유저 100명이 좋아할 만한 영화의 목록을 출력

In [37]:
%time Aliens_recom_result = get_recommendation_by_title('Aliens', 100,10)
Aliens_recom_result

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 4.06 s


[array(['The Lord of the Rings: The Fellowship of the Ring',
        'The Lord of the Rings: The Two Towers',
        'The Lord of the Rings: The Return of the King', 'Deadpool',
        'Guardians of the Galaxy',
        'Harry Potter and the Prisoner of Azkaban', 'The Dark Knight',
        'Toy Story 3', 'Shaun of the Dead', 'Hot Fuzz'], dtype=object),
 array(['Memento', 'The Lord of the Rings: The Fellowship of the Ring',
        'The Lord of the Rings: The Return of the King',
        'The Lord of the Rings: The Two Towers', 'The Dark Knight',
        'Gone Girl', 'Inception', 'Kill Bill: Vol. 1', 'Toy Story 3',
        'The Departed'], dtype=object),
 array(['The Dark Knight', 'Inception', 'Gladiator',
        'The Dark Knight Rises',
        'The Lord of the Rings: The Fellowship of the Ring',
        'The Matrix Reloaded', 'The Lord of the Rings: The Two Towers',
        'The Lord of the Rings: The Return of the King', 'Interstellar',
        'Memento'], dtype=object),
 array(['

# Content-Based Recommendation

In [38]:
movies_df = pd.read_csv(input_path + "tmdb_5000_movies.csv")
credits_df = pd.read_csv(input_path + "tmdb_5000_credits.csv")

# drop columns
# budget, homepage, original_language, original_title, production_countries, release_date,
# revenue, spoken_languages, status
movies_df = movies_df.drop(columns=['budget', 'homepage', 'original_language', 'original_title', 'production_countries',
                                    'release_date', 'revenue', 'spoken_languages', 'status', 'runtime', 'tagline'])

credits_df = credits_df.drop(columns=['title'])
credits_df.columns=['id', 'cast', 'crew']

In [39]:
id_list_1 = list()
for item in credits_df['id']:
    if type(item) is not int:
        print('Wrong Id data type')
    id_list_1.append(item)
print(len(id_list_1))

for item in movies_df['id']:
    if item not in id_list_1:
        print('Id is not sufficient : ', item)

4803


In [40]:
total_df = movies_df.join(credits_df.set_index('id'), on='id')
total_df.head(1)

,genres,id,keywords,overview,popularity,production_companies,title,vote_average,vote_count,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [41]:
total_df = total_df.drop_duplicates(['title'], keep='first')
total_df = pd.DataFrame(total_df.values, columns=['genres', 'id', 'keywords', 'overview', 'popularity', 'production_companies', 'title', 'vote_average', 'vote_count', 'cast', 'crew'])
#k.drop_duplicates(subset=['A'], keep=False)

In [42]:
import json

json_columns = ['genres', 'keywords', 'cast', 'crew', 'production_companies']
for column in json_columns:
    total_df[column] = total_df[column].apply(json.loads)

In [43]:
count=0
for title, crew_list in zip(total_df['title'], total_df["crew"]):
    if not crew_list:
        count+=1
print(count)

28


In [44]:
director_list = list()
director_set = set()
for title, crew_list in zip(total_df['title'], total_df['crew']):
    temp = list()
    for one_crew_dict in crew_list:
        if one_crew_dict['job'] == 'Director':
            temp.append(one_crew_dict['name'])
            director_set.add(one_crew_dict['name'])
    director_list.append(temp)

In [45]:
director_dict = dict()
director_dict = dict.fromkeys(director_set)
count = 0
for key in director_dict.keys():
    director_dict[key] = count
    count += 1
director_rev_dict = {v:k for k,v in director_dict.items()}
print(len(director_dict.keys()))

2575


In [46]:
director_vector = list()
director_columns_name = list()

# Director vectorize - one hot encoding
for directors in director_list:
    temp_director = [0]*len(director_dict.keys())
    for director in directors:
        temp_director[director_dict[director]] = 1
    director_vector.append(temp_director)

# Generate column names of Director dataframe
for count in range(len(director_dict.keys())):
    director_columns_name.append('Director' + str(count))
    
print(len(director_vector))
Directors = pd.DataFrame(director_vector, columns=director_columns_name)

4800


In [47]:
pro_com_set = set()
com_list_list = list()
for company_list in total_df['production_companies']:
    temp_com_list = list()
    for company in company_list:
        pro_com_set.add(company['name'])
        temp_com_list.append(company['name'])
    com_list_list.append(temp_com_list)

pro_com_vectors = list()
pro_com_dict = dict()
pro_com_dict = dict.fromkeys(pro_com_set)
count = 0
for keyitem in pro_com_dict.keys():
    pro_com_dict[keyitem] = count
    count += 1

for com_list in com_list_list:
    temp_vec = [0]*5017
    for com in com_list:
        temp_vec[pro_com_dict[com]] = 1
    pro_com_vectors.append(temp_vec)

production_com_columns = list()
for i in range(5017):
    temp_name = 'company' + str(i)
    production_com_columns.append(temp_name)

Production_Companies = pd.DataFrame(pro_com_vectors, columns = production_com_columns)
#Production_Companies['title'] = total_df['title']

In [48]:
# 한 영화가 가진 가장 많은 production company의 종류 개수 : 26
max_len = 0
for com_list in com_list_list:
    if len(com_list) > max_len:
        max_len = len(com_list)
print(max_len)

26


In [49]:
# 한 영화가 가진 가장 적은 production company의 종류 개수 : 0
min_len = 9999
for com_list in com_list_list:
    if len(com_list) < min_len:
        min_len = len(com_list)
print(min_len)

0


In [50]:
genre_set = set()
genre_list_list = list()

for genre_list in total_df['genres']:
    temp_genre = list()
    for genre in genre_list:
        genre_set.add(genre['name'])
        temp_genre.append(genre['name'])
    genre_list_list.append(temp_genre)
print(len(genre_set))
print(genre_set)

20
{'Foreign', 'Documentary', 'Science Fiction', 'Drama', 'Comedy', 'Mystery', 'Music', 'Romance', 'Family', 'Western', 'Animation', 'War', 'Action', 'Crime', 'Thriller', 'History', 'TV Movie', 'Fantasy', 'Adventure', 'Horror'}


In [51]:
genre_dict = dict()
genre_dict = dict.fromkeys(genre_set)
count = 0
for key_item in genre_dict.keys():
    genre_dict[key_item] = count
    count += 1
genre_vector = list()
for genre_list in genre_list_list:
    temp_genre_vec = [0] * 20
    for genre in genre_list:
        temp_genre_vec[genre_dict[genre]] = 1
    genre_vector.append(temp_genre_vec)
genre_cols = list()
for i in range(20):
    temp_genre = 'genre' + str(i)
    genre_cols.append(temp_genre)

Genre = pd.DataFrame(genre_vector, columns=genre_cols)

In [52]:
#Genre.head(10)

In [53]:
# 한 영화가 가진 가장 많은 genre의 종류 개수 : 7
max_len = 0
for genre_list in genre_list_list:
    if len(genre_list) > max_len:
        max_len = len(genre_list)
print(max_len)

7


In [54]:
# 한 영화가 가진 가장 적은 genre의 종류 개수 : 0
min_len = 9999
for genre_list in genre_list_list:
    if len(genre_list) < min_len:
        min_len = len(genre_list)
print(min_len)

0


In [55]:
def vectorize(value_set, list_list, col_name):
    feature_vector = list()
    feature_dict = dict.fromkeys(value_set)
    feature_columns = list()
    count = 0
    
    for key in feature_dict.keys():
        feature_dict[key] = count
        count+=1
    for one_line in list_list:
        temp_list = [0]*len(value_set)
        for one_ele in one_line:
            temp_list[feature_dict[one_ele]] = 1
        feature_vector.append(temp_list)
    for i in range(len(value_set)):
        temp_name = str(col_name) + str(i)
        feature_columns.append(temp_name)
        
    result = pd.DataFrame(feature_vector, columns = feature_columns)
    return result

In [56]:
keywords=total_df['keywords']
keywords_list_list = list()
keywords_set = set()

for keywords_list in keywords:
    temp_list = list()
    for keyword in keywords_list:
        temp_list.append(keyword['name'])
        keywords_set.add(keyword['name'])
    keywords_list_list.append(temp_list)
    
Keywords = vectorize(keywords_set, keywords_list_list, 'keyword')
Keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Columns: 9809 entries, keyword0 to keyword9808
dtypes: int64(9809)
memory usage: 359.2 MB


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')

total_df['overview'] = total_df['overview'].fillna('')

# construct TF-IDF matrix by fitting and transforming the overview data
tfidf_matrix = tfidf.fit_transform(total_df['overview'])

# output the shape of tfidf_matrix
tfidf_matrix.shape

(4800, 20967)

In [58]:
#import scipy
# overview 학습의 결과를 unpack 해보았다.
#M1 = scipy.sparse.diags(tfidf_matrix.A.ravel()).tocsr()
#M1

In [59]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(total_df.index, index=total_df['title']).drop_duplicates()

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, target_count=10, cosine_sim=cosine_similarity):
    result_dict = dict()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:1 + target_count] # except for itself
    movies_indices = [i[0] for i in sim_scores]
    for index, score in zip(movies_indices, sim_scores):
        print("index:", score[0],"  ","score:", score[1], "   ", total_df['title'].iloc[index])
    #return total_df['title'].iloc[movies_indices]

In [60]:
cast_all = dict()
for cast_group in total_df['cast']:
    for cast in cast_group:
        if cast['id'] in cast_all:
            cast_all[cast['id']][1] += 1
        else:
            cast_all[cast['id']] = [cast['name'], 1]

In [61]:
len(cast_all)

54557

In [62]:
cast_freq = list(cast_all.values())
cast_freq.sort(reverse=True, key=lambda x: (x[1]))

In [63]:
cast2vec_dict = dict()
count = 0;
for cast_id in cast_all.keys():
    cast2vec_dict[cast_id] = count
    count += 1

cast_vectors = list()
for cast_group in total_df['cast']:
    cast_vector = [0] * len(cast_all)
    for cast in cast_group:
        cast_vector[cast2vec_dict[cast['id']]] = 1
    cast_vectors.append(cast_vector)

In [64]:
len(cast_vectors[0])

54557

In [65]:
cast_col = list()
count = 0
for ele in cast_vectors[0]:
    name = 'cast' + str(count)
    count += 1
    cast_col.append(name)

In [66]:
Cast = pd.DataFrame(cast_vectors, columns=cast_col)

In [67]:
Cast.shape

(4800, 54557)

In [68]:
print('-'*62)
print("-"*20 + "Vectorization Complete" + "-"*20)
print("Director shape : ", Directors.shape)
print("Genre shape", Genre.shape)
print("Production Companies shape : ", Production_Companies.shape)
print("Cast shape : ", Cast.shape)
print("Keyword shape : ", Keywords.shape)
print('-'*62)

--------------------------------------------------------------
--------------------Vectorization Complete--------------------
Director shape :  (4800, 2575)
Genre shape (4800, 20)
Production Companies shape :  (4800, 5017)
Cast shape :  (4800, 54557)
Keyword shape :  (4800, 9809)
--------------------------------------------------------------


## Calculate Similarity

In [69]:
from scipy import spatial


def Similarity(movie_title1, movie_title2, Feature):
    movie1_row = Feature.iloc[indices[movie_title1]]
    movie2_row = Feature.iloc[indices[movie_title2]]
    
    Dist = spatial.distance.cosine(movie1_row, movie2_row)
    if math.isnan(Dist):
        #print('nan!')
        Dist = 1.0
    return 1 - Dist

In [70]:
Similarity("The Avengers", "Thor", Genre)

0.6666666666666667

In [71]:
def get_overview_score(movie_title1, movie_title2):
    score = 0
    idx_1 = indices[movie_title1]
    idx_2 = indices[movie_title2]
    score = cosine_similarity[idx_1][idx_2]
    return score

In [72]:
get_overview_score("The Avengers", "Batman")

0.0584644599388013

In [73]:
def total_reccommendation_v2_cosDist(movie_title1, movie_title2):
    genre_score = Similarity(movie_title1, movie_title2, Genre)
    cast_score = Similarity(movie_title1, movie_title2, Cast)
    director_score = Similarity(movie_title1, movie_title2, Directors)
    pro_com_score = Similarity(movie_title1, movie_title2, Production_Companies)
    overview_score = get_overview_score(movie_title1, movie_title2)
    keywords_score = Similarity(movie_title1, movie_title2, Keywords)
    #total_score = genre_score + cast_score + director_score + pro_com_score + (10*overview_score) + (5*keywords_score)
    """
    print('-'*60)
    print('Genre : ', genre_score)
    print('Cast : ', cast_score)
    print('Director : ', director_score)
    print('Production Companies : ', pro_com_score)
    print('Overview : ', overview_score)
    print('Keywords : ', keywords_score)
    """
    return genre_score, cast_score, director_score, pro_com_score, overview_score, keywords_score

In [74]:
total_reccommendation_v2_cosDist("Iron Man", "Iron Man 3")

(1.0, 0.0725515215453656, 0.0, 1.0, 0.15088869151876816, 0.5773502691896257)

In [75]:
total_reccommendation_v2_cosDist("Batman", "Batman")

(1.0, 1.0, 1.0, 1.0, 1.0000000000000002, 1.0)

In [76]:
def get_score_lists_of_movies(target_movie_title):   
    
    total_genre = list()
    total_director = list()
    total_overview = list()
    total_keywords = list()
    total_cast = list()
    total_procom = list()
    
    for title in total_df.title:
        #score_list.append(total_reccommendation_v2_cosDist(target_movie_title, title))
        a,b,c,d,e,f = total_reccommendation_v2_cosDist(target_movie_title, title)
        total_genre.append(a)
        total_cast.append(b)
        total_director.append(c)
        total_procom.append(d)
        total_overview.append(e)
        total_keywords.append(f)
    
    return [total_genre, total_director, total_overview, total_keywords, total_cast, total_procom]

In [77]:
def regularize(one_dim_list):
    mean = np.mean(one_dim_list, axis=0)
    std = np.std(one_dim_list, axis=0)
    result = list()
    for item in one_dim_list:
        result.append((item-mean)/std)
    return result

In [78]:
final_genre = list()
final_director = list()
final_overview = list()
final_keywords = list()
final_cast = list()
final_procom = list()

def total_recommendation_v2(movie_title, target_count):
    score_lists = get_score_lists_of_movies(movie_title)
    score_dict = dict()
    global final_genre
    global final_director
    global final_overview
    global final_keywords
    global final_cast
    global final_procom
    
    regularized_genre_score = regularize(score_lists[0])
    final_genre = regularized_genre_score
    regularized_director_score = regularize(score_lists[1])
    final_director = regularized_director_score
    regularized_overview_score = regularize(score_lists[2])
    final_overview = regularized_overview_score
    regularized_keywords_score = regularize(score_lists[3])
    final_keywords = regularized_keywords_score
    regularized_cast_score = regularize(score_lists[4])
    final_cast = regularized_cast_score
    regularized_procom_score = regularize(score_lists[5])
    final_procom = regularized_procom_score
    
    for idx, title in enumerate(list(total_df.title)):
        score_dict[title] = regularized_genre_score[idx] + regularized_director_score[idx]\
        + regularized_overview_score[idx] + regularized_keywords_score[idx]\
        + regularized_cast_score[idx] + regularized_procom_score[idx]
    
    sorted_score = sorted(score_dict.items(), key=lambda kv: kv[1], reverse = True)
    return sorted_score[1:1 + target_count]

In [79]:
%time recom_result = total_recommendation_v2("Aliens", 10)
recom_result

C:\Anaconda3\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Wall time: 1min 38s


[('Avatar', 39.634658652354084),
 ('Alien³', 37.28663163739932),
 ('True Lies', 36.69004190632586),
 ('The Abyss', 36.4214812636759),
 ('Alien', 36.0040147123586),
 ('The Terminator', 34.07983949201465),
 ('Alien: Resurrection', 33.868256788524015),
 ('Titanic', 32.929787934409404),
 ('Terminator 2: Judgment Day', 30.184929805300207),
 ('AVP: Alien vs. Predator', 19.856653153557843)]

In [80]:
def regularized_similarity_score(movie_title2):
    idx = indices[movie_title2]
    print('-'*60)
    print('Genre : ', final_genre[idx])
    print('Cast : ', final_cast[idx])
    print('Director : ', final_director[idx])
    print('Production Companies : ', final_procom[idx])
    print('Overview : ', final_overview[idx])
    print('Keywords : ', final_keywords[idx])
    

In [81]:
def get_similarity_percent(movie_title, res_count=100, sample_count=100):
    contents_recom_list = [x[0] for x in total_recommendation_v2(movie_title, res_count)]
    count = 0
    for results in get_recommendation_by_title(movie_title, sample_count, res_count):
        for movie in contents_recom_list:
            if movie in results:
                count +=1
    print(count)

In [82]:
def movie_existence(movietitle):
    try:
        movie_id = movieId_list.index[movieId_list['title'] == movietitle].tolist()[0]
        return True
    except IndexError as e:
        return False


In [83]:
movie_existence("asdfsaasf")

False

## final_function 은 두 가지 결과를 포함한 list를 리턴한다.
- ### 인자로 넘어온 title과 가장 비슷한 영화
- ### 인자로 넘어온 title을 좋아하는 사람들이 가장 선호하는 영화

In [84]:
from collections import Counter

def final_function(movietitle, sample_size = 1, recommendation_length = 1, userId=0):
    if not movie_existence(movietitle):
         print("Input movie does not exist in dataset.")
    most_similar_movie = total_recommendation_v2(movietitle, 1)[0][0]
    most_funnable_movies = get_recommendation_by_title(movietitle, sample_size, recommendation_length)
    recom_list = list()
    for items in most_funnable_movies:
        for item in items:
            recom_list.append(item)
    if userId == 0:
        most_funnable_result = Counter(recom_list).most_common()[:10]
    else:
        user_already_seen = result[result['movieId'].isin(ratings_df[ratings_df['userId'] == userId]['movieId'])].title
        #print(user_already_seen)
        most_funnable_result = Counter([x for x in recom_list if x not in user_already_seen.values]).most_common()[0][0]
        
    return [most_similar_movie, most_funnable_result]

In [85]:
#%time final_function("The Dark Knight",100,5)

In [86]:
#%time final_function("The Avengers",100,5)

In [87]:
#%time final_function("The Terminator", 100, 5)

In [88]:
#%time final_function("Saving Private Ryan", 100, 5)

In [89]:
#%time final_function("Star Wars",100,5)

In [90]:
#%time final_function("Indiana Jones and the Last Crusade",100,5)

In [91]:
#%time final_function("The Fifth Element",100,5)

## 전체 결과 출력 - Collaborative filtering 에서 계속 뻔한 결과가 나오는 문제가 발생
- ### 입력으로 받은 userId가 본 영화는 추천하지 않도록 변경
- ### 어느정도 해결되었지만 아직 뻔한 영화가 추천되는 경향이 남음
- ### 영화 출시년도 정보를 추가하기로 결정

In [92]:
#%time final_function("Aliens",100,5)

## Collaborative filtering에서 2000년도 이후 출시된 영화만 추천하기로 결정 

In [93]:
Id_title_year_over2000

,movieId,title,release_date
2,6377,Finding Nemo,2003
5,3910,Dancer in the Dark,2000
8,6539,Pirates of the Caribbean: The Curse of the Bla...,2003
9,6874,Kill Bill: Vol. 1,2003
12,54272,The Simpsons Movie,2007
13,7361,Eternal Sunshine of the Spotless Mind,2004
14,4235,Amores perros,2000
15,45722,Pirates of the Caribbean: Dead Man's Chest,2006
16,37733,A History of Violence,2005
18,5810,8 Mile,2002


In [94]:
def final_function_v2(movietitle, sample_size = 1, recommendation_length = 1, userId=0):
    if not movie_existence(movietitle):
         print("Input movie does not exist in dataset.")
    most_similar_movie = total_recommendation_v2(movietitle, 1)[0][0]
    most_funnable_movies = get_recommendation_by_title(movietitle, sample_size, recommendation_length)
    recom_list = list()
    for items in most_funnable_movies:
        for item in items:
            recom_list.append(item)
    if userId == 0:
        most_funnable_result = Counter(recom_list).most_common()[:10]
    else:
        user_already_seen = result[result['movieId'].isin(ratings_df[ratings_df['userId'] == userId]['movieId'])].title
        #print(user_already_seen)
        most_funnable_result = Counter([x for x in recom_list if x not in user_already_seen.values]).most_common()[0][0]
        
    return [most_similar_movie, most_funnable_result]

In [95]:
%time final_function_v2('Interstellar', 100, 10, 231)

C:\Anaconda3\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 1min 43s


['The Dark Knight Rises', 'Whiplash']